<h3>Stp1: Install packages and load modules </h3>

In [ ]:
#!pip install xgboost
#!pip install statistics

In [79]:
import xgboost as xgb
import pandas as pd
import statistics
from scipy.stats import mannwhitneyu as MUT
from scipy.stats import fisher_exact as FSH
from collections import defaultdict
import random

<h3>Data loading and preparation</h3>
<p> Load the data file. the data files should be placed under <b><font color="Fuchsia" size="3" >./RBL_70_30/</font></b> refer to the notebook directory</p

In [80]:
dataPath = "./RBL_70_30/"
rawAccountData70 = dataPath + "raw_account_70.csv"
rawAccountData30 = dataPath + "raw_account_30.csv"
rawData70 = dataPath + "raw_data_70.csv"
rawData30 = dataPath + "raw_data_30.csv"
rawEnquiry70 = dataPath + "raw_enquiry_70.csv"
rawEnquiry30 = dataPath + "raw_enquiry_30.csv"

In [81]:
################################# load inside the historical data ################
df_rawAccountData70 = pd.read_csv(rawAccountData70)
df_rawAccountData30 = pd.read_csv(rawAccountData30)
df_rawData70 = pd.read_csv(rawData70)
df_rawData30 = pd.read_csv(rawData30)
df_rawEnquiry70 = pd.read_csv(rawEnquiry70)
df_rawEnquiry30 = pd.read_csv(rawEnquiry30)


In [82]:
frames = [df_rawAccountData30, df_rawAccountData70]
df_rawAccountData= pd.concat(frames, ignore_index=True)

frames = [df_rawData30, df_rawData70]
df_rawData= pd.concat(frames, ignore_index=True)

frames = [df_rawEnquiry30, df_rawEnquiry70]
df_rawEnquiry= pd.concat(frames, ignore_index=True)

In [83]:
# incase there are duplicates
#df_rawData = df_rawData.drop_duplicates(subset="apprefno", keep="last")

In [84]:
############################# Processing categorical data ########################
#### account data
ac_cat_cols = ["member_short_name", "acct_type", \
            "owner_indic", "writtenoffandsettled", "typeofcollateral", "paymentfrequency"]
ac_sparse2cat_cols = ["valueofcollateral", "writtenoffamounttotal", \
                   "writtenoffamountprincipal", "settlementamount", "dateofentryforcibilremarkscode"]
ac_date_cols = ["dt_opened", "upload_dt", "opened_dt", \
             "last_paymt_dt", "closed_dt", "reporting_dt", "paymt_str_dt", "paymt_end_dt"]
ac_cont_cols = ["high_credit_amt", "cur_balance_amt", "amt_past_due", \
                "creditlimit", "cashlimit", "rateofinterest", "repaymenttenure", "actualpaymentamount"]
#### enquiry data
eq_cat_cols = ["member_short_name", "enq_purpose"]
eq_date_cols = ["dt_opened", "upload_dt", "enquiry_dt"]
eq_cont_cols = ["enq_amt"]
#### current status
cu_cat_cols = ["card_name", "promo_code", "aip_status", "app_disposition", \
               "status_type", "approved_credit_limit", "intl_trn", "fee_code", "override_fee_code", \
               "override_months", "acq_source", "se_code", "lead_code", "mktg_code", "app_gender", \
               "mob_verified", "marital_status", "edu_qualification", "app_res_city", "res_type", \
               "permanent_same", "employment_type", "company_type", "industry_type", "designation", \
               "months_exp", "office_city", "id_proof", "existing_bank", "rbl_relationship_no", \
               "app_has_card", "existing_card_issuer", "app_title", "app_state", "app_existing_other_loan_cc", \
               "app_existing_rbl_cust", "app_international_transaction_fl", "app_si_ecs_flg", \
               "app_resident_india", "app_perm_state", "app_perm_city", "app_perm_country", "app_employment_type", \
               "app_office_state", "app_pref_mailing_addr", "app_go_green", "app_addon_card_dob", \
               "app_addon_card_relationship", "app_memo_status", "app_fcu_check_status", "worst_dpd6"]
cu_date_cols = ["entry_time", "cibil_datetime", "app_dob", "existing_card_start_date"]
cu_cont_cols = ["cibil_score", "num_dependents", "res_from_yr", "res_from_month", \
                "net_monthly_income", "year_joining", "years_exp", "existing_credit_limit", \
                "app_gross_monthly_income"]
cu_sparse2cat_cols = ["reject_reason_code", "reject_reason_desc", "cin", "app_coupon_code"]

In [85]:
def build_id_group(df, key_col, tag_col):
    id_tag = dict(zip(df[key_col], df[tag_col]))
    print id_tag.items()[0:10]
    return id_tag
################
key_col = "apprefno"
tag_col = "bad_flag_worst6"
id_tag = build_id_group(df_rawData, key_col, tag_col)
print id_tag["CCR0HH9N0"]

[('CC8IRED11', 0), ('CCG420NGY9', 0), ('CCI9MI20N3', 0), ('CC9A8JAR01', 0), ('CCAA446L3I', 0), ('CCMR59M7D2', 0), ('CCPAA6P348', 0), ('CC9IT6V3A', 0), ('CCDR79EW8', 0), ('CCA5I2A88A', 0)]
1


In [86]:
def random_sampler(k, lst):
    if len(lst) < k:
        return lst
    else:
        return [ lst[i] for i in sorted(random.sample(xrange(len(lst)), k)) ]

In [87]:
############## processing of catgorial data ################### --only for binary
def extracte_historical_cat_and_ft_sele(df, cols, key_col, min_num, id_tag, critical_val, suf):
    id_cat_fq = defaultdict(lambda:defaultdict(int))
    cat_ids = defaultdict(set)
    num_rcds = len(df[df.keys()[0]])
    pfids = set()
    err_ct = 0
    for i in range(num_rcds):
        uid = df[key_col][i]
        try:
            pfids.add(str(uid))
        except:
            print uid, type(uid), "[" + uid + "]", i
            err_ct += 1
            if err_ct > 50:
                return
            continue
        for col in cols: 
            val = df[col][i]
            if val == None:
                continue
            cat_key = suf + "@#_" + col + "@#" + str(val)
            id_cat_fq[uid][cat_key] += 1
            cat_ids[cat_key].add(uid)
    print "toal categoricals to be scanned: ", len(cat_ids)
    uids = list(pfids)
    ########## discard sparse data and non-onformatie features ############
    cats_keep = set()
    for cat in cat_ids:
        if len(cat_ids[cat]) < min_num:
            print "Skip\t[" + cat + "]", len(cat_ids[cat])
            continue
        o_vals = []
        z_vals = []
        for uid in uids:
            if uid not in id_tag:
                continue
            tag = id_tag[uid]
            if tag not in [0,1]:
                continue
            if tag == 1:
                o_vals.append(id_cat_fq[uid][cat])
            else:
                z_vals.append(id_cat_fq[uid][cat])
        if min(len(z_vals), len(o_vals)) < min_num:
            continue
        try:
            v, p = MUT(random_sampler(200, o_vals), random_sampler(200, z_vals))
        except:
            continue
        #print v, p, cat
        if p <= critical_val:
            print "Catgorical\t[" + cat + "] is informative: p =", p, ">>", len(z_vals), len(o_vals)
            cats_keep.add(cat)
    print "total kept categories", len(cats_keep)
    col_dic = defaultdict(list)
    
    col_dic["userid"] = uids
    
    for cat in cats_keep:
        temp = []
        for uid in uids:
            temp.append(id_cat_fq[uid][cat])
        col_dic[cat] = temp
    return col_dic

In [88]:
key_col = "apprefno"
min_num = 20
critical_val = 0.05

In [89]:
suf = "acc"
ac_cat_colDic = extracte_historical_cat_and_ft_sele(df_rawAccountData, ac_cat_cols, key_col, \
                                                 min_num, id_tag, critical_val, suf)

toal categoricals to be scanned:  60
Skip	[acc@#_acct_type@#16] 1
Skip	[acc@#_writtenoffandsettled@#99.0] 8
Catgorical	[acc@#_typeofcollateral@#nan] is informative: p = 0.0172227330655 >> 32670 1466
Skip	[acc@#_writtenoffandsettled@#7.0] 2
Catgorical	[acc@#_paymentfrequency@#nan] is informative: p = 2.64572202481e-06 >> 32670 1466
Skip	[acc@#_acct_type@#33] 7
Skip	[acc@#_acct_type@#34] 14
Skip	[acc@#_acct_type@#36] 1
Catgorical	[acc@#_acct_type@#10] is informative: p = 2.29461126791e-05 >> 32670 1466
Skip	[acc@#_acct_type@#11] 6
Skip	[acc@#_member_short_name@#RBL BANK] 3
Skip	[acc@#_acct_type@#14] 8
Catgorical	[acc@#_acct_type@#15] is informative: p = 0.0282005062764 >> 32670 1466
Skip	[acc@#_acct_type@#56] 11
Skip	[acc@#_acct_type@#57] 1
Catgorical	[acc@#_acct_type@#54] is informative: p = 0.0415031306076 >> 32670 1466
Catgorical	[acc@#_writtenoffandsettled@#nan] is informative: p = 2.97766502549e-05 >> 32670 1466
Skip	[acc@#_typeofcollateral@#3.0] 19
Skip	[acc@#_member_short_name@#RB

In [90]:
suf = "enq"
eq_cat_colDic = extracte_historical_cat_and_ft_sele(df_rawEnquiry, eq_cat_cols, key_col, \
                                                 min_num, id_tag, critical_val, suf)

toal categoricals to be scanned:  43
Skip	[enq@#_enq_purpose@#57.0] 9
Skip	[enq@#_enq_purpose@#11.0] 15
Catgorical	[enq@#_enq_purpose@#10.0] is informative: p = 0.04077124042 >> 32670 1466
Skip	[enq@#_enq_purpose@#55.0] 12
Skip	[enq@#_enq_purpose@#43.0] 6
Catgorical	[enq@#_enq_purpose@#6.0] is informative: p = 0.00557135585207 >> 32670 1466
Skip	[enq@#_enq_purpose@#42.0] 3
Skip	[enq@#_enq_purpose@#58.0] 8
Catgorical	[enq@#_enq_purpose@#5.0] is informative: p = 0.00621609100297 >> 32670 1466
Skip	[enq@#_enq_purpose@#41.0] 12
Catgorical	[enq@#_enq_purpose@#32.0] is informative: p = 0.0415031306076 >> 32670 1466
Skip	[enq@#_enq_purpose@#31.0] 6
Skip	[enq@#_enq_purpose@#59.0] 13
Catgorical	[enq@#_enq_purpose@#7.0] is informative: p = 0.0275949746691 >> 32670 1466
Skip	[enq@#_enq_purpose@#56.0] 17
Skip	[enq@#_enq_purpose@#36.0] 7
Skip	[enq@#_enq_purpose@#33.0] 18
Skip	[enq@#_enq_purpose@#16.0] 2
Skip	[enq@#_enq_purpose@#34.0] 5
Skip	[enq@#_enq_purpose@#15.0] 14
total kept categories 5


In [91]:
########################## continus data historical: counts min max median #####################
def extracte_historical_val(df, cols, key_col, min_num, id_tag, critical_val, suf):
    id_col_vals = defaultdict(lambda:defaultdict(list))
    
    col_ids = defaultdict(set)
    num_rcds = len(df[df.keys()[0]])
    pfids = set()
    
    for i in range(num_rcds):
        uid = df[key_col][i]
        try:
            pfids.add(uid)
        except:
            print uid
            return
        for col in cols:
            if pd.isnull(df[col][i]):
                continue
            try:
                val = float(str(df[col][i]).replace("\"", ""))
            except:
                print df[col][i], col
                return None
            id_col_vals[uid][col].append(val)
            col_ids[col].add(uid)
    
    uids = list(pfids)
    ########## discard minor features ############
    col_keep = set()
    for col in col_ids:
        if len(col_ids[col]) < min_num:
            print "Skip\t[" + col + "]", len(col_ids[col])
            continue
        col_keep.add(col)
    print "total kept categories at stage 1", len(col_keep)
    col_dic = defaultdict(list)
    col_dic["userid"] = uids
    keep_ct = 0
    for col in col_keep:
        temp_max = []
        temp_min = []
        temp_med = []
        temp_cnt = []
        for uid in uids:
            vals = id_col_vals[uid][col]
            if len(vals) == 0:
                minv = -99
                maxv = -99
                medv = -99
                ct = 0
            else:
                minv = min(vals)
                maxv = max(vals)
                medv = statistics.median(vals)
                ct = len(vals)
            temp_max.append(maxv)
            temp_min.append(minv)
            temp_med.append(medv)
            temp_cnt.append(ct)
        #------------ test
        max_o_vals = []
        max_z_vals = []
        min_o_vals = []
        min_z_vals = []
        med_o_vals = []
        med_z_vals = []
        cnt_o_vals = []
        cnt_z_vals = []
        for i, uid in enumerate(uids):
            if uid not in id_tag:
                continue
            tag = id_tag[uid]
            if tag not in [0, 1]:
                continue
            if tag == 1:
                max_o_vals.append(temp_max[i])
                min_o_vals.append(temp_min[i])
                med_o_vals.append(temp_med[i])
                cnt_o_vals.append(temp_cnt[i])    
            else:
                max_z_vals.append(temp_max[i])
                min_z_vals.append(temp_min[i])
                med_z_vals.append(temp_med[i])
                cnt_z_vals.append(temp_cnt[i])
        
        v, p = MUT(random_sampler(200, max_o_vals), random_sampler(100, max_z_vals))
        if p <= critical_val:
            col_str = suf + "@#_" + col +"_max"
            col_dic[col_str] = temp_max
            keep_ct += 1
            print "Continous\t[" + col_str + "] is informative: p =", p
        v, p = MUT(random_sampler(200, min_o_vals), random_sampler(100, min_z_vals))
        if p <= critical_val:
            col_str = suf + "@#_" + col +"_min"
            col_dic[col_str] = temp_min
            keep_ct += 1
            print "Continous\t[" + col_str + "] is informative: p =", p
        v, p = MUT(random_sampler(200, med_o_vals), random_sampler(100, med_z_vals))
        if p <= critical_val:
            col_str = suf + "@#_" + col +"_med"
            col_dic[col_str] = temp_med
            keep_ct += 1
            print "Continous\t[" + col_str + "] is informative: p =", p
        v, p = MUT(random_sampler(200, cnt_o_vals), random_sampler(100, cnt_z_vals))
        if p <= critical_val:
            col_str = suf + "@#_" + col +"_cnt"
            col_dic[col_str] = temp_cnt
            keep_ct += 1
            print "Continous\t[" + col_str + "] is informative: p =", p
    print "total kept features:", keep_ct
    return col_dic
################################


In [92]:
suf = "acc"
ac_cnt_colDic = extracte_historical_val(df_rawAccountData, ac_cont_cols, \
                                        key_col, min_num, id_tag, critical_val, suf)

total kept categories at stage 1 8
Continous	[acc@#_rateofinterest_cnt] is informative: p = 0.0351814712458
Continous	[acc@#_amt_past_due_cnt] is informative: p = 0.0158650587972
Continous	[acc@#_cashlimit_med] is informative: p = 0.0399023899025
Continous	[acc@#_cashlimit_cnt] is informative: p = 0.00110861883824
Continous	[acc@#_creditlimit_max] is informative: p = 0.00038054695492
Continous	[acc@#_creditlimit_min] is informative: p = 0.0245892223653
Continous	[acc@#_creditlimit_cnt] is informative: p = 0.0127109555915
Continous	[acc@#_actualpaymentamount_med] is informative: p = 0.0253313925375
total kept features: 8


In [93]:
suf = "enq"
eq_cnt_colDic = extracte_historical_val(df_rawEnquiry, eq_cont_cols, key_col, min_num, id_tag, critical_val, suf)

total kept categories at stage 1 1
total kept features: 0


In [94]:
def convert_to_dates(raw_date_str, refer_date):
    try:
        comp = raw_date_str.split("-")
        date_str = "".join(comp[0:2]) + "20" + comp[2]
        date = pd.to_datetime(date_str, format = "%d%b%Y")
        return (date - refer_date).days
    except:
        return None

In [95]:
def is_informative(uids, vals, refs, id_tags, critical_val, col_str):
    try:
        o_vals = []
        z_vals = []
        for i, uid in enumerate(uids):
            if refs[i] > 99998:
                continue
            tag = id_tags[uid]
            if tag not in [1, 0]:
                continue
            if tag == 1:
                o_vals.append(vals[i])
            else:
                z_vals.append(vals[i])
        v, p = MUT(random_sampler(200, o_vals), random_sampler(200, z_vals))
        if p <= critical_val:
            print "data_field\t[" + col_str + "] is informative: p =", p
            return True
        else:
            return False  
    except:
        return False

In [96]:
def time_interval(df, col_key, time_cols, critical_val, id_tags, suf):
    num_rcds = len(df[col_key])
    print "number of records:", num_rcds
    print "num_cols to be processed:", len(time_cols)
    refer_date = pd.to_datetime("1 1 2014", format = "%d %m %Y")
    uidst = set()
    for i in range(num_rcds):
        uidst.add(df[col_key][i])
    uids = list(uidst)
    print "Number of uids:", len(uids)
    colDict = defaultdict(list)
    num_kp = 0
    colDict["userid"] = uids
    
    for time_col in time_cols:
        print "processing>", time_col
        uid_days = defaultdict(list)
        for i in range(num_rcds):
            if i % 50000 == 0:
                print i
            date_str = df[time_col][i]
            days = convert_to_dates(date_str, refer_date) # can be none
            if days == None:
                continue
            uid = df[col_key][i]
            uid_days[uid].append(days)
        
        #print uid_days.items()[0:10]
        
        print "stag1.....on", len(uids), "users"
        ############ get earliest, latest time, min, max, median, intervals, counts, density ###########
        maxItvs = []
        minItvs = []
        medItvs = []
        starts = []
        ends = []
        cts = []
        denss = []
        count = 0
        for uid in uids:
            if count%5000 == 0:
                print count
            #if count > 10:
            #    return None
            count += 1
            if len(uid_days[uid]) <= 1:
                start = 0
                en = 99999
                ct = 1
                maxItv = 99999 - start
                minItv = 99999 - start
                medItv = 99999 - start
                dens = 1.0/max(float(en - start), 1)
                #print "--1"
            else:
                #print "----2"
                tms = sorted(uid_days[uid])
                start = tms[0]
                en = tms[-1]
                ct = len(tms)
                itvs = []
                for i in range(1, ct):
                    itvs.append(tms[i] - tms[i-1])
                maxItv = max(itvs)
                minItv = min(itvs)
                medItv = statistics.median(itvs)
                dens = float(ct)/max(float(en - start), 1)
                #print "--2"
            #print maxItv, minItv, minItv, medItv, start, en, ct, dens
            maxItvs.append(maxItv)
            minItvs.append(minItv)
            medItvs.append(medItv)
            starts.append(start)
            ends.append(en)
            cts.append(ct)
            denss.append(dens)
            #if count > 4:
                #break
        #break
        print "stg2...."
        col_str = suf + "@#_" + time_col + "@#maxItv"
        if is_informative(uids, maxItvs, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = maxItvs
            num_kp += 1
        
        col_str = suf + "@#_" + time_col + "@#minItv"
        if is_informative(uids, minItvs, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = minItvs
            num_kp += 1
        
        col_str = suf + "@#_" + time_col + "@#medItv"
        if is_informative(uids, medItvs, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = medItvs
            num_kp += 1
            
        col_str = suf + "@#_" + time_col + "@#counts"
        if is_informative(uids, cts, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = cts
            num_kp += 1
            #print cts[0:20]
            #return None
            
        col_str = suf + "@#_" + time_col + "@#dens"
        if is_informative(uids, denss, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = denss
            num_kp += 1
            
        col_str = suf + "@#_" + time_col + "@#starts"
        if is_informative(uids, starts, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = starts
            num_kp += 1
        
        col_str = suf + "@#_" + time_col + "@#ends"
        if is_informative(uids, ends, maxItvs, id_tags, critical_val, col_str):
            colDict[col_str] = ends
            num_kp += 1    
        #################
        
    print "Total kept features:", num_kp    
    return colDict
#####################
suf = "acc"
ac_dateColDict = time_interval(df_rawAccountData, key_col, ac_date_cols, critical_val, id_tag, suf)


number of records: 265897
num_cols to be processed: 8
Number of uids: 34136
processing> dt_opened
0
50000
100000
150000
200000
250000
stag1.....on 34136 users
0
5000
10000
15000
20000
25000
30000
stg2....
data_field	[acc@#_dt_opened@#counts] is informative: p = 0.0249157150163
data_field	[acc@#_dt_opened@#dens] is informative: p = 0.000516268226839
processing> upload_dt
0
50000
100000
150000
200000
250000
stag1.....on 34136 users
0
5000
10000
15000
20000
25000
30000
stg2....
data_field	[acc@#_upload_dt@#counts] is informative: p = 0.000965005025269
data_field	[acc@#_upload_dt@#dens] is informative: p = 0.0327416931624
processing> opened_dt
0
50000
100000
150000
200000
250000
stag1.....on 34136 users
0
5000
10000
15000
20000
25000
30000
stg2....
data_field	[acc@#_opened_dt@#minItv] is informative: p = 0.0010224646792
data_field	[acc@#_opened_dt@#medItv] is informative: p = 0.0160598715249
data_field	[acc@#_opened_dt@#counts] is informative: p = 0.0060898082843
data_field	[acc@#_opened_d

In [97]:
print ac_dateColDict.keys()

['acc@#_upload_dt@#dens', 'acc@#_upload_dt@#counts', 'acc@#_dt_opened@#counts', 'acc@#_opened_dt@#counts', 'acc@#_paymt_end_dt@#counts', 'userid', 'acc@#_opened_dt@#minItv', 'acc@#_last_paymt_dt@#minItv', 'acc@#_opened_dt@#medItv', 'acc@#_dt_opened@#dens', 'acc@#_paymt_end_dt@#maxItv', 'acc@#_paymt_end_dt@#ends', 'acc@#_reporting_dt@#minItv', 'acc@#_last_paymt_dt@#ends', 'acc@#_reporting_dt@#counts', 'acc@#_opened_dt@#dens']


In [98]:
suf = "enq"
eq_dateColDict = time_interval(df_rawEnquiry, key_col, eq_date_cols, critical_val, id_tag, suf)

number of records: 586687
num_cols to be processed: 3
Number of uids: 34136
processing> dt_opened
0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
stag1.....on 34136 users
0
5000
10000
15000
20000
25000
30000
stg2....
processing> upload_dt
0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
stag1.....on 34136 users
0
5000
10000
15000
20000
25000
30000
stg2....
data_field	[enq@#_upload_dt@#counts] is informative: p = 0.00367716414159
processing> enquiry_dt
0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
stag1.....on 34136 users
0
5000
10000
15000
20000
25000
30000
stg2....
data_field	[enq@#_enquiry_dt@#maxItv] is informative: p = 0.00475556469847
data_field	[enq@#_enquiry_dt@#dens] is informative: p = 0.0169673870221
data_field	[enq@#_enquiry_dt@#starts] is informative: p = 0.0427999544812
data_field	[enq@#_enquiry_dt@#ends] is informative: p = 0.00321879796029
Total kept features: 5


In [99]:
######################  none aggregated convertion and feture selection ################

In [100]:
def is_informative2(uids, vals, id_tags, critical_val, col_str):
    o_vals = []
    z_vals = []
    for i, uid in enumerate(uids):
        if vals[i] == None:
            continue
        tag = id_tags[uid]
        if tag not in [1, 0]:
            continue
        if tag == 1:
            o_vals.append(vals[i])
        else:
            z_vals.append(vals[i])
    v, p = MUT(random_sampler(200, o_vals), random_sampler(200, z_vals))
    if p <= critical_val:
        print "date_field\t[" + col_str + "] is informative: p =", p
        return True
    else:
        return False  

In [101]:
#df_rawData70
def convert_date_to_relative_and_ft_selection(df, col_key, time_cols, critical_val, id_tags, suf):
    num_rcds = len(df[col_key])
    print "number of records:", num_rcds
    print "num_cols to be processed:", len(time_cols)
    refer_date = pd.to_datetime("1 1 2014", format = "%d %m %Y")
    uidst = set()
    for i in range(num_rcds):
        uidst.add(df[col_key][i])
    uids = list(uidst)
    print "Number of uids:", len(uids)
    colDict = defaultdict(list)
    num_kp = 0
    colDict["userid"] = uids
    
    for time_col in time_cols:
        print "processing>", time_col
        uid_days = defaultdict(int)
        for i in range(num_rcds):
            if i % 50000 == 0:
                print i
            date_str = df[time_col][i]
            days = convert_to_dates(date_str, refer_date) # can be none
            #if days == None:
            #    continue
            uid = df[col_key][i]
            uid_days[uid] = days

        print "stg1...."
        col_str = suf + "@#_" + time_col + "@#_rela"
        if is_informative2(uids, df[time_col], id_tags, critical_val, col_str):
            daysl = []
            for uid in uids:
                if uid not in uid_days:
                    daysl.append(None)
                else:
                    daysl.append(uid_days[uid])
            colDict[col_str] = daysl
            num_kp += 1
        #################
        
    print "Total kept features:", num_kp    
    return colDict
#####################
suf = "cur"
cu_dateColDict = convert_date_to_relative_and_ft_selection(df_rawData, key_col, cu_date_cols, \
                                                           critical_val, id_tag, suf)


number of records: 34136
num_cols to be processed: 4
Number of uids: 34136
processing> entry_time
0
stg1....
processing> cibil_datetime
0
stg1....
processing> app_dob
0
stg1....
processing> existing_card_start_date
0
stg1....
Total kept features: 0


In [102]:
def is_informative_fisher(uids, vals, id_tags, critical_val, col_str):
    cat_zo_ct = defaultdict(lambda: defaultdict(int))
    for i, uid in enumerate(uids):
        if vals[i] == None:
            continue
        cat = vals[i]
        tag = id_tags[uid]
        if (tag not in [1, 0]) or (cat not in [1, 0]):
            continue
        cat_zo_ct[cat][tag] += 1
        
    bg = [cat_zo_ct[0][0], cat_zo_ct[0][1]]
    cat = [cat_zo_ct[1][0], cat_zo_ct[1][1]]
    #try:
    print ">>", bg, cat
    v, p = FSH([bg, cat])
    if p <= critical_val:
        print cat, "of date_field\t[" + col_str + "] is informative: p =", p
        return True
    else:
        return False
    #except:
    #    print "Warning: data error"
    #    return False

In [103]:
def non_agg_cat_processing(df, col_key, cat_cols, critical_val, id_tags, min_sz, suf):
    num_rcds = len(df[col_key])
    print "number of records:", num_rcds
    print "num_cols to be processed:", len(cat_cols)
    uidst = set()
    for i in range(num_rcds):
        uidst.add(df[col_key][i])
    uids = list(uidst)
    print "Number of uids:", len(uids)
    colDict = defaultdict(list)
    num_kp = 0
    colDict["userid"] = uids
    
    uid_cat_val = defaultdict(lambda: defaultdict(int))
    print "#### stage 1 #################"
    cat_ct = defaultdict(int)
    for i in range(num_rcds):
        if i % 50000 == 0:
            print i
        uid = df[col_key][i]
        for cat_col in cat_cols:
            if df[cat_col][i] != None:
                cat = suf + "@#_" + cat_col + "@#" + str(df[cat_col][i])
                uid_cat_val[uid][cat] = 1
                cat_ct[cat] += 1
    print "#### stage 2 #################"
    for cat in cat_ct:
        if cat_ct[cat] < min_sz:
            continue
        print "processing>", cat
        vals = []
        for uid in uids:
            val = uid_cat_val[uid][cat]
            vals.append(val)
            
        if is_informative_fisher(uids, vals, id_tags, critical_val, cat):
            colDict[cat] = vals
            num_kp += 1
        ##############
        
    print "Total kept features:", num_kp    
    return colDict
#####################
suf = "cur"
cu_catColDict = non_agg_cat_processing(df_rawData, key_col, cu_cat_cols, critical_val, id_tag, 100, suf)

number of records: 34136
num_cols to be processed: 51
Number of uids: 34136
#### stage 1 #################
0
#### stage 2 #################
processing> cur@#_approved_credit_limit@#70000.0
>> [32433, 1453] [237, 13]
processing> cur@#_res_type@#Self
>> [17909, 824] [14761, 642]
processing> cur@#_approved_credit_limit@#71000.0
>> [32447, 1449] [223, 17]
processing> cur@#_edu_qualification@#Diploma
>> [32365, 1457] [305, 9]
processing> cur@#_acq_source@#SARE
>> [31847, 1413] [823, 53]
[823, 53] of date_field	[cur@#_acq_source@#SARE] is informative: p = 0.0139430038332
processing> cur@#_existing_bank@#State Bank of India
>> [32140, 1445] [530, 21]
processing> cur@#_approved_credit_limit@#126000.0
>> [32356, 1454] [314, 12]
processing> cur@#_existing_bank@#Punjab National Bank
>> [32550, 1463] [120, 3]
processing> cur@#_se_code@#HA01
>> [32538, 1461] [132, 5]
processing> cur@#_acq_source@#MA14
>> [32529, 1452] [141, 14]
[141, 14] of date_field	[cur@#_acq_source@#MA14] is informative: p = 0.

processing> cur@#_industry_type@#KPO/LPO/ITES
>> [32378, 1457] [292, 9]
processing> cur@#_app_go_green@#N
>> [31554, 1401] [1116, 65]
[1116, 65] of date_field	[cur@#_app_go_green@#N] is informative: p = 0.0406505117244
processing> cur@#_app_go_green@#Y
>> [29753, 1345] [2917, 121]
processing> cur@#_se_code@#VRP4
>> [32511, 1460] [159, 6]
processing> cur@#_se_code@#VRP1
>> [32465, 1455] [205, 11]
processing> cur@#_se_code@#VRP2
>> [32511, 1461] [159, 5]
processing> cur@#_designation@#SR EXECUTIVE
>> [31870, 1438] [800, 28]
processing> cur@#_approved_credit_limit@#79000.0
>> [32466, 1449] [204, 17]
[204, 17] of date_field	[cur@#_approved_credit_limit@#79000.0] is informative: p = 0.0187168070403
processing> cur@#_marital_status@#2.0
>> [25135, 1067] [7535, 399]
[7535, 399] of date_field	[cur@#_marital_status@#2.0] is informative: p = 0.000310815339454
processing> cur@#_acq_source@#SA14
>> [32537, 1450] [133, 16]
[133, 16] of date_field	[cur@#_acq_source@#SA14] is informative: p = 0.00070

[218, 17] of date_field	[cur@#_approved_credit_limit@#82000.0] is informative: p = 0.0342067931288
processing> cur@#_designation@#MANAGER
>> [30472, 1395] [2198, 71]
[2198, 71] of date_field	[cur@#_designation@#MANAGER] is informative: p = 0.00375871233808
processing> cur@#_app_perm_city@#56.0
>> [32545, 1463] [125, 3]
processing> cur@#_promo_code@#VNTX
>> [30925, 1376] [1745, 90]
processing> cur@#_app_res_city@#Kolhapur
>> [32567, 1465] [103, 1]
processing> cur@#_app_existing_rbl_cust@#N
>> [654, 11] [32016, 1455]
[32016, 1455] of date_field	[cur@#_app_existing_rbl_cust@#N] is informative: p = 0.000215392535071
processing> cur@#_approved_credit_limit@#156000.0
>> [32513, 1463] [157, 3]
processing> cur@#_app_pref_mailing_addr@#R
>> [11437, 471] [21233, 995]
[21233, 995] of date_field	[cur@#_app_pref_mailing_addr@#R] is informative: p = 0.025014519078
processing> cur@#_app_pref_mailing_addr@#O
>> [21256, 996] [11414, 470]
[11414, 470] of date_field	[cur@#_app_pref_mailing_addr@#O] is in

processing> cur@#_rbl_relationship_no@#nan
>> [156, 3] [32514, 1463]
processing> cur@#_se_code@#S203
>> [32558, 1463] [112, 3]
processing> cur@#_se_code@#S202
>> [32545, 1463] [125, 3]
processing> cur@#_promo_code@#TTTX
>> [31973, 1420] [697, 46]
[697, 46] of date_field	[cur@#_promo_code@#TTTX] is informative: p = 0.013244313994
processing> cur@#_approved_credit_limit@#72000.0
>> [32454, 1451] [216, 15]
processing> cur@#_approved_credit_limit@#98000.0
>> [32391, 1459] [279, 7]
processing> cur@#_designation@#SUPERVISOR
>> [32517, 1453] [153, 13]
[153, 13] of date_field	[cur@#_designation@#SUPERVISOR] is informative: p = 0.0329251912427
processing> cur@#_existing_bank@#nan
>> [7420, 351] [25250, 1115]
processing> cur@#_edu_qualification@#nan
>> [27693, 1273] [4977, 193]
[4977, 193] of date_field	[cur@#_edu_qualification@#nan] is informative: p = 0.0307450506026
processing> cur@#_edu_qualification@#MBA/MMS
>> [32363, 1453] [307, 13]
processing> cur@#_approved_credit_limit@#99000.0
>> [323

[379, 27] of date_field	[cur@#_acq_source@#CD01] is informative: p = 0.02561953563
processing> cur@#_approved_credit_limit@#149000.0
>> [32546, 1461] [124, 5]
processing> cur@#_app_state@#17.0
>> [31940, 1421] [730, 45]
[730, 45] of date_field	[cur@#_app_state@#17.0] is informative: p = 0.0391953581652
processing> cur@#_designation@#SOFTWARE ENGINEER
>> [32554, 1462] [116, 4]
processing> cur@#_app_res_city@#Chennai
>> [32401, 1456] [269, 10]
processing> cur@#_app_has_card@#N
>> [17401, 556] [15269, 910]
[15269, 910] of date_field	[cur@#_app_has_card@#N] is informative: p = 1.04883182013e-30
processing> cur@#_app_has_card@#Y
>> [15292, 911] [17378, 555]
[17378, 555] of date_field	[cur@#_app_has_card@#Y] is informative: p = 1.05967974985e-30
processing> cur@#_office_city@#Vadodara
>> [32114, 1447] [556, 19]
processing> cur@#_company_type@#MNC
>> [30614, 1372] [2056, 94]
processing> cur@#_approved_credit_limit@#35000.0
>> [31602, 1404] [1068, 62]
processing> cur@#_id_proof@#Driving Licens

processing> cur@#_existing_bank@#Yes Bank
>> [32514, 1458] [156, 8]
processing> cur@#_approved_credit_limit@#141000.0
>> [32524, 1460] [146, 6]
processing> cur@#_office_city@#Mumbai
>> [30889, 1397] [1781, 69]
processing> cur@#_approved_credit_limit@#124000.0
>> [32453, 1461] [217, 5]
processing> cur@#_app_res_city@#Kanchipuram
>> [32562, 1464] [108, 2]
processing> cur@#_approved_credit_limit@#125000.0
>> [32372, 1454] [298, 12]
processing> cur@#_se_code@#H013
>> [32542, 1463] [128, 3]
processing> cur@#_approved_credit_limit@#63000.0
>> [32361, 1451] [309, 15]
processing> cur@#_se_code@#M001
>> [32546, 1463] [124, 3]
processing> cur@#_office_city@#Bangalore
>> [31948, 1422] [722, 44]
processing> cur@#_office_city@#Coimbatore
>> [32563, 1466] [107, 0]
[107, 0] of date_field	[cur@#_office_city@#Coimbatore] is informative: p = 0.0153765331102
processing> cur@#_se_code@#RM03
>> [32547, 1459] [123, 7]
processing> cur@#_card_name@#Titanium Deligh
>> [24433, 1145] [8237, 321]
[8237, 321] of d

In [104]:
def get_the_cont_data_fields(df, cols, key_col, suf):
    num_rcds = len(df[key_col])
    colDict = defaultdict(list)
    print "num_uid to be processed: ", num_rcds
    for i in range(num_rcds):
        if i % 100000 == 0:
            print i
        uid = df[key_col][i]
        colDict["userid"].append(uid)
        for col in cols:
            colDict[suf + "@#_" + col].append(df[col][i])
    return colDict

In [105]:
suf = "cur"
cu_contColDict = get_the_cont_data_fields(df_rawData, cu_cont_cols, key_col, suf)

num_uid to be processed:  34136
0


In [106]:
def train_test_dict(df_rawData70, key_col):
    uid_trainYes = defaultdict(int)
    num_rcds = len(df_rawData70[key_col])
    for i in range(num_rcds):
        uid = str(df_rawData70[key_col][i])
        uid_trainYes[uid] = 1
    
    num_rcds = len(df_rawData30[key_col])
    for i in range(num_rcds):
        uid = str(df_rawData30[key_col][i])
        uid_trainYes[uid] = 0
    return uid_trainYes
############3333
uid_trainYes = train_test_dict(df_rawData70, key_col)
print uid_trainYes.items()[0:3]

[('CC33ANGG46', 1), ('CCG420NGY9', 1), ('CCI9MI20N3', 1)]


In [107]:
dics = [ac_cat_colDic, eq_cat_colDic, ac_cnt_colDic, eq_cnt_colDic, ac_dateColDict, \
        eq_dateColDict, cu_dateColDict, cu_catColDict, cu_contColDict]

In [115]:
def integrate_cols(colDicts, uid_trainYes, key_col, uid_tag):
    pos_colDict = defaultdict(list)
    neg_colDict = defaultdict(list)
    
    uid_col_val = defaultdict(lambda:defaultdict(str))
    col_set = set()
    for k, colDict in enumerate(colDicts):
        #print k
        cols = [x for x in colDict.keys() if x != "apprefno"]
        col_set = col_set.union(cols)
        num_rcds = len(colDict[key_col])
        #print num_rcds
        for i in range(num_rcds):
            uid = colDict[key_col][i]
            for col in cols:
                val0 = colDict[col][i]
                try:
                    val = float(val0)
                except:
                    val == "-99"
                uid_col_val[uid][col] = str(val)
            #if i > 20:
            #    return None, None
    
    print "into_single colDict..."
    allColDict_train = defaultdict(list)
    allColDict_test = defaultdict(list)
    col_list = list(col_set)
    
    
    print col_list[0:10], "..."
    tr_uids = []
    ts_uids = []
    tr_posneg = []
    ts_posneg = []
    ct = 0
    skp_ct = 0
    print "number of uids to be processed", len(uid_trainYes.items())
    for uid, tag in uid_trainYes.items():
        if ct % 10000 == 0:
            print ct
        ct += 1
        if (uid not in uid_trainYes) or (uid not in uid_tag):
            skp_ct +=1
            continue
            
        pso_neg_tag = uid_tag[uid]
        if tag == 1:
            tr_uids.append(uid)
            tr_posneg.append(pso_neg_tag)
            for col in col_list:
                allColDict_train[col].append(uid_col_val[uid][col])
        else:
            ts_uids.append(uid)
            ts_posneg.append(pso_neg_tag)
            for col in col_list:
                allColDict_test[col].append(uid_col_val[uid][col])
    
    allColDict_train["userid"] = tr_uids
    allColDict_test["userid"] = ts_uids
    allColDict_train["PosNgTag"] = tr_posneg
    allColDict_test["PosNgTag"] = ts_posneg
    print "skipped:", skp_ct
    return allColDict_train, allColDict_test
################### test ##################33
allColDict_train, allColDict_test = integrate_cols(dics, uid_trainYes, "userid", id_tag)

into_single colDict...
['cur@#_worst_dpd6@#9', 'cur@#_se_code@#AS18', 'cur@#_year_joining', 'acc@#_actualpaymentamount_med', 'cur@#_worst_dpd6@#0', 'cur@#_se_code@#AS11', 'cur@#_app_office_state@#13.0', 'enq@#_upload_dt@#counts', 'cur@#_card_name@#Titanium Deligh', 'cur@#_app_state@#10.0'] ...
number of uids to be processed 34136
0
10000
20000
30000
skipped: 0


In [116]:
df_test = pd.DataFrame(allColDict_test)
df_train = pd.DataFrame(allColDict_train)

In [117]:
df_test.to_csv("./tranfered_data/test.csv")
df_train.to_csv("./tranfered_data/train.csv")

In [111]:
print cu_contColDict.keys()

['cur@#_years_exp', 'cur@#_year_joining', 'cur@#_app_gross_monthly_income', 'cur@#_existing_credit_limit', 'cur@#_cibil_score', 'userid', 'cur@#_res_from_month', 'cur@#_num_dependents', 'cur@#_res_from_yr', 'cur@#_net_monthly_income']
